In [1]:
import json
import pandas as pd

from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [2]:
#location = "boston"
location = "geneva"
#location = "hong-kong"
index_name = "airbnb_history_" + location

In [3]:
from elasticsearch.helpers import scan

res = scan(
    es,
    index=index_name,
    query={"query": { "match_all" : {}}}
)

In [4]:
data = list(res)

In [5]:
len(data)

128927

In [6]:
# appending rows 
docs = []
for item in data:  
    doc = {
        'id': item['_id'],
        'index': item['_index'],
        'listing_url': item['_source']['listing_url'],
        'scrape_id': item['_source']['scrape_id'],
        'last_scraped': item['_source']['last_scraped'],
        'name': item['_source']['name'],
        'host_id': item['_source']['host_id'],
        'price': item['_source']['price'],
        'availability_30': item['_source']['availability_30'],
        'availability_60': item['_source']['availability_60'],
        'availability_90': item['_source']['availability_90'],
        'availability_365': item['_source']['availability_365'],
        'first_review': item['_source']['first_review'],
        'last_review': item['_source']['last_review'],
        'review_scores_rating': item['_source']['review_scores_rating'],
        'review_scores_accuracy': item['_source']['review_scores_accuracy'],
        'review_scores_cleanliness': item['_source']['review_scores_cleanliness'],
        'review_scores_checkin': item['_source']['review_scores_checkin'],
        'review_scores_communication': item['_source']['review_scores_communication'],
        'review_scores_location': item['_source']['review_scores_location'],
        'review_scores_value': item['_source']['review_scores_value'],
        'overall_rating': item['_source']['overall_rating']
    }
    
    docs.append(doc) 

In [7]:
len(docs)

128927

In [8]:
def divide_chunks(docs, n): 
    """ Split list of documents into n-sized chunks
    :param l: list
    :param n: num of paritions
    """
    org_size = len(docs)
    chunk_size = org_size / float(n)
    print(org_size)
    print(chunk_size)
    
    return [ docs[int(round(chunk_size * i)): int(round(chunk_size * (i + 1)))] for i in range(n) ]

In [9]:
partitions = divide_chunks(docs, 3)

128927
42975.666666666664


In [10]:
len(partitions[0])

42976

In [11]:
partitions[0][:2]

[{'id': 'nNheP3cBYxiStLCpI4Vf',
  'index': 'airbnb_history_geneva',
  'listing_url': 'https://www.airbnb.com/rooms/11694938',
  'scrape_id': 20160909025305,
  'last_scraped': '20160909',
  'name': 'Geneve maison',
  'host_id': 1762307,
  'price': 350.0,
  'availability_30': 29,
  'availability_60': 59,
  'availability_90': 89,
  'availability_365': 244,
  'first_review': '19910101',
  'last_review': '0',
  'review_scores_rating': '0',
  'review_scores_accuracy': '0',
  'review_scores_cleanliness': '0',
  'review_scores_checkin': '0',
  'review_scores_communication': '0',
  'review_scores_location': '0',
  'review_scores_value': '0',
  'overall_rating': 0.0},
 {'id': 'ndheP3cBYxiStLCpI4Vf',
  'index': 'airbnb_history_geneva',
  'listing_url': 'https://www.airbnb.com/rooms/8959271',
  'scrape_id': 20160909025305,
  'last_scraped': '20160909',
  'name': 'Cozy room in a nice Villa area',
  'host_id': 31906729,
  'price': 45.0,
  'availability_30': 12,
  'availability_60': 34,
  'availabili

In [12]:
len(partitions[1])

42975

In [13]:
len(partitions[2])

42976

In [14]:
import json
i = 0

for p in partitions:
    
    with open('airbnb_' + location + "_" + str(i) + '.json', 'w') as f:
        json.dump(p, f)
        
    i = i+1